In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

from caen_felib import device

## Connect

In [ ]:
dig = device.connect('dig2://caendgtz-eth-16218.caen.local/')

## Reset

In [ ]:
dig.cmd.reset()

## Example of node browsing

In [ ]:

par_name = 'cupver'
print(f'Info of parameter {par_name}:')

for child in dig.par.get_node(f'/{par_name}').child_nodes:
    print(f'\t{child.name} == {child.value}')

## Configure digitizer

In [ ]:
dig.set_value('/par/AcqTriggerSource', 'SwTrg')
dig.set_value('/par/RecordLengthS', '1024')
dig.set_value('/par/PreTriggerS', '128')

for ch in dig.ch.child_nodes:
	ch.set_value(f'/par/DCOffset', f'{0 + int(ch.name)}')

dig.set_value('/endpoint/par/activeendpoint', 'scope')

## Configure endpoint

In [ ]:
nch = int(dig.par.numch.value)
reclen = int(dig.par.recordlengths.value)

data_format = [
	{
		'name': 'TIMESTAMP',
		'type': 'U64',
	},
	{
		'name': 'WAVEFORM',
		'type': 'U16',
		'dim': 2,
		'shape': [nch, reclen],
	},
	{
		'name': 'WAVEFORM_SIZE',
		'type': 'U64',
		'dim': 1,
		'shape': [nch],
	},
]

# set_read_data_format returns allocated buffers
data = dig.endpoint.scope.set_read_data_format(data_format)


## Test waveform plot

In [ ]:
# Start acquisition
dig.cmd.armacquisition()
dig.cmd.swstartacquisition()

dig.cmd.sendswtrigger()
dig.endpoint.scope.read_data(-1, data)

dig.cmd.disarmacquisition()

# Print timestamp
print(f'Timestamp: {data[0].value}')

# Plot waveforms
for ch_waveform in data[1].value:
    plt.plot(ch_waveform)

plt.show()